In [ ]:
import cv2
from typing import Any
import subprocess
import os
from cv2.typing import MatLike
from argparse import ArgumentParser
import numpy as np
from mirage.mirage_helpers import *
from mirage.pose_extract_base import MLAbstractInterface
from mirage.rgb_interface import CameraInterface
from mirage.movenet import MovenetInterface
from mirage.skeleton import SkeletonDetection

%matplotlib inline
from matplotlib import pyplot as plt

def showim(im):
    plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
    plt.show()

def show_images(images, cols = 1, titles = None):
    """Display a list of images in a single figure with matplotlib.
    
    Parameters
    ---------
    images: List of np.arrays compatible with plt.imshow.
    
    cols (Default = 1): Number of columns in figure (number of rows is 
                        set to np.ceil(n_images/float(cols))).
    
    titles: List of titles corresponding to each image. Must have
            the same length as titles.
    """
    assert((titles is None)or (len(images) == len(titles)))
    n_images = len(images)
    if titles is None: titles = ['Image (%d)' % i for i in range(1,n_images + 1)]
    fig = plt.figure()
    for n, (image, title) in enumerate(zip(images, titles)):
        a = fig.add_subplot(cols, int(np.ceil(n_images/float(cols))), n + 1)
        if image.ndim == 2:
            plt.gray()
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        a.set_title(title)
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_images)
    plt.show()

In [ ]:
import yaml
def ost_to_np(filename):
    with open(filename) as s:
        try:
            q = yaml.safe_load(s)
            return {
                "image_width": q["image_width"],
                "image_height": q["image_height"],
                "camera_name": q["camera_name"],
                "camera_matrix": np.array(q["camera_matrix"]["data"]).reshape(
                    (q["camera_matrix"]["rows"], q["camera_matrix"]["cols"])
                ),
                "distortion_coefficients": np.array(q["distortion_coefficients"]["data"]).reshape(
                    (q["distortion_coefficients"]["rows"], q["distortion_coefficients"]["cols"])
                ),
            }
        except yaml.YAMLError as exc:
            print(exc)
            return None


def process_and_viz_split(i_cam: CameraInterface, ml_interface: MLAbstractInterface) -> MatLike:
    img = i_cam.get_next_frame()
    img_a, img_b = split_image_stack(img)
    a_crop = determine_crop_region(s_a, img_a.shape[0], img_a.shape[1])
    b_crop = determine_crop_region(s_b, img_b.shape[0], img_b.shape[1])
    img_a_kp = ml_interface.predict(img_a, a_crop)
    img_b_kp = ml_interface.predict(img_b, b_crop)
    s_a.update_predictions(img_a_kp, (1.0 / i_cam.get_frame_rate_per_second()))
    s_b.update_predictions(img_b_kp, (1.0 / i_cam.get_frame_rate_per_second()))
    img_a_viz = skeleton_to_image(img_a, s_a)
    img_b_viz = skeleton_to_image(img_b, s_b)
    return stack_image(img_a_viz, img_b_viz)


def process_and_viz(i_cam: CameraInterface, ml_interface: MLAbstractInterface) -> MatLike:
    img = i_cam.get_next_frame()
    img_kp = ml_interface.predict(img)
    s_a.update_predictions(img_kp, (1.0 / i_cam.get_frame_rate_per_second()))
    img_viz = skeleton_to_image(img, s_a)
    return img_viz


In [ ]:
input_file = "../../Data/Reference_Parayno_Jeru.mp4"
frame_number_start = 150
frame_number_end = 160
frames = []
splitimage = True

In [ ]:
# main
ml_interface = MovenetInterface()

In [ ]:
s_a = SkeletonDetection()
s_b = SkeletonDetection()
i_cam = CameraInterface(input_file)
i_cam.set_frame(frame_number_start)
frame_number_end = frame_number_end if frame_number_end != 0 else i_cam.get_total_frames()
##### reset

In [ ]:
def display_log_info(im: MatLike, skele: SkeletonDetection):
    img = im.copy()
    num_joints = len(skele.joints)
    print(num_joints)
    starting_y = 150
    starting_x = im.shape[1]-350
    height_per_text = 30
    img = overlay_rect(img, starting_x - 30, 20, im.shape[1] - starting_x+20, im.shape[0] - 40)
    for j in skele.joints.values():
        out = f"{j.name}: {j.confidence*100:.0f}%"
        img = cv2.putText(img, f"{out:<20}", (starting_x, starting_y), cv2.FONT_HERSHEY_PLAIN, 2, (255,0,255), 2)
        starting_y += height_per_text
    return img

def overlay_rect(img: MatLike, x: int, y: int, w: int, h: int):
    # First we crop the sub-rect from the image
    sub_img = img[y:y+h, x:x+w]
    white_rect = np.ones(sub_img.shape, dtype=np.uint8) * 0
    res = cv2.addWeighted(sub_img, 0.25, white_rect, 0.75, 1.0)
    # Putting the image back to its position
    img[y:y+h, x:x+w] = res
    return img

In [ ]:
img = i_cam.get_next_frame()
img_a, img_b = split_image_stack(img)
a_crop = determine_crop_region(s_a, img_a.shape[0], img_a.shape[1])
b_crop = determine_crop_region(s_b, img_b.shape[0], img_b.shape[1])

print(a_crop)
if a_crop is not None:
    a_crop_img = crop_image(img_a, int(a_crop["y_min"]),
                    int(a_crop["height"]),
                    int(a_crop["x_min"]),
                    int(a_crop["width"]),
                    0,)
else:
    a_crop_img = img_a * 0
img_a_kp = ml_interface.predict(img_a, a_crop)
s_a.update_predictions(img_a_kp, (1.0 / i_cam.get_frame_rate_per_second()))
img_a_viz = skeleton_to_image(img_a, s_a)

show_images([img_a, a_crop_img, img_a_viz], 2, ["original image", "crop used", "Estimated points"])
# showim(display_log_info(img_a, s_a))

In [ ]:
color_list = [tuple(np.random.randint(0,255,3).tolist()) for j in range(100)]
def drawlines(img1,img2,lines,pts1,pts2):
 ''' img1 - image on which we draw the epilines for the points in img2
 lines - corresponding epilines '''
 r,c = img1.shape
 img1 = cv2.cvtColor(img1,cv2.COLOR_GRAY2BGR)
 img2 = cv2.cvtColor(img2,cv2.COLOR_GRAY2BGR)
 for i, (r,pt1,pt2) in enumerate(zip(lines,pts1,pts2)):
     color = color_list[i%len(color_list)]
     x0,y0 = map(int, [0, -r[2]/r[1] ])
     x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
     img1 = cv2.line(img1, (x0,y0), (x1,y1), color,1)
     img1 = cv2.circle(img1,tuple(np.int32(pt1)),5,color,-1)
     img2 = cv2.circle(img2,tuple(np.int32(pt2)),5,color,-1)
 return img1,img2

def drawpoints(img1, pts1):
    r,c = img1.shape
    img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    for i, pt in enumerate(pts1):
        color = color_list[i%len(color_list)]
        im1 = cv2.circle(img1, tuple(np.int32(pt)), 9, color, 3)
    return img1

    
# pts1 = np.float64(pts1)
# pts2 = np.float64(pts2)
# E, mask = cv2.findEssentialMat(pts1, pts2, cam_a_intr["camera_matrix"], cam_a_intr["distortion_coefficients"], cam_b_intr["camera_matrix"], cam_b_intr["distortion_coefficients"])

In [ ]:
# img = i_cam.get_next_frame()
img_a, img_b = split_image_stack(img)
img_a = cv2.cvtColor( img_a, cv2.COLOR_BGR2GRAY)
img_b = cv2.cvtColor( img_b, cv2.COLOR_BGR2GRAY)
cam_b_intr = ost_to_np("cama_ost.yaml")
cam_a_intr = ost_to_np("camb_ost.yaml")

img_a_undistorted = cv2.undistort(img_a, cam_a_intr["camera_matrix"], cam_a_intr["distortion_coefficients"], None, None)
img_b_undistorted = cv2.undistort(img_b, cam_b_intr["camera_matrix"], cam_b_intr["distortion_coefficients"], None, None)

sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(img_a_undistorted, None)
kp2, des2 = sift.detectAndCompute(img_b_undistorted, None)

FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)
pts1 = []
pts2 = []

for i, (m,n) in enumerate(matches):
    if m.distance < 0.7*n.distance:
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)


pts1 = np.int32(pts1)
pts2 = np.int32(pts2)
F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_8POINT)
pts1 = pts1[mask.ravel()==1]
pts2 = pts2[mask.ravel()==1]

lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1, 1, 2), 2, F)
lines1 = lines1.reshape(-1, 3)
img_a_pts = drawpoints(img_a, pts1)
img_a_epi, _ = drawlines(img_a, img_b, lines1, pts1, pts2)

lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1, 1, 2), 1, F)
lines2 = lines2.reshape(-1, 3)
img_b_pts = drawpoints(img_b, pts2)
img_b_epi, _ = drawlines(img_b, img_a, lines2, pts2, pts1)


print(F)
show_images([img_a, img_b, img_a_undistorted, img_b_undistorted, img_a_pts, img_b_pts, img_a_epi, img_b_epi],
            4, 
            ["original image a", "original image b", "undist A", "undist B", "points a", "points b", "epi a", "epi b"])

In [ ]:
print(cam_a_intr["camera_matrix"])
print(cam_b_intr["camera_matrix"])

In [ ]:
print(E)

In [ ]:
[f"{m} :::: {n}" for (m, n) in zip(pts1, pts2)]